# Multi-Scenario MORDM Gelderland
This Jupyter Notebook uses the model developed by [Ciullo et al. (2019)](https://scholar.google.com/citations?hl=en&user=fDZCVVYAAAAJ&view_op=list_works&sortby=pubdate#d=gs_md_cita-d&u=%2Fcitations%3Fview_op%3Dview_citation%26hl%3Den%26user%3DfDZCVVYAAAAJ%26sortby%3Dpubdate%26citation_for_view%3DfDZCVVYAAAAJ%3AqjMakFHDy7sC%26tzom%3D-120) and the [ema_workbench by Jan Kwakkel](https://github.com/quaquel/EMAworkbench) to evaluate possible strategies for the Gelderland province to increase its resilience to flood risk. It is specifically aimed towards finding optimal strategies for the Gelderland province, namely minimizing casualties and damages across the model runtime in dike rings 1-3 and minimizing dike investment costs for all dike rings. 

## Imports

In [42]:
# Standard Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import networkx as nx
import scipy as sp
import plotly.express as px


# Workbench Imports
from ema_workbench import (Model, MultiprocessingEvaluator, ScalarOutcome, RealParameter, IntegerParameter, CategoricalParameter, optimize, Scenario, Constant, ema_logging,perform_experiments, SequentialEvaluator, Policy)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator
from ema_workbench.analysis import parcoords, prim


ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>

## General Model Setup
Here, we specify the numbers for the experiment and optimization runs to perform.

In [2]:
# Number of nfes for the Model
n_nfe = 20000

#Number of scenarios for the deep uncertainty evaluation of promising scenarios
n_scenarios = 1000

#Path to the reference scenarios to use for MORDM
path_noaction = "../results/10000Scenarios_NoAction_PF1.tar.gz"

## Model Specification

In [3]:
# Model Imports
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation

dike_model, planning_steps = get_model_for_problem_formulation(6)
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

## Specify reference scenario
Firstly, we load the results from the open exploration with no action taken to select a reference scenario that shows the most expected number of deaths. This, so-to-say worst-case scenario is then used to find the optimal policies using the MORDM approach.  

In [4]:
experiments, outcomes = load_results(path_noaction)

[MainProcess/INFO] results loaded succesfully from /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/10000Scenarios_NoAction_PF1.tar.gz


In [5]:
outcomes_df = pd.DataFrame(outcomes)

#Get index of worst scenario
index_wc = outcomes_df.sort_values("Expected Number of Deaths").tail(1).index
experiment_wc = experiments.iloc[index_wc]
reference_scenarios = [Scenario(f"{index}", **row) for index, row in experiment_wc.iloc[0:,0:19].iterrows()]

## Run Optimization on Worst Case Reference Scenario

In [6]:
ema_logging.log_to_stderr(ema_logging.INFO)

convergence_metrics = [EpsilonProgress()]

with MultiprocessingEvaluator(dike_model) as evaluator:
    results_epsilon_0_1 = evaluator.optimize(nfe=n_nfe, epsilons=[0.1,]*len(dike_model.outcomes), 
    reference = reference_scenarios[0], convergence= convergence_metrics, searchover="levers")

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/20000 nfe
[MainProcess/INFO] generation 5: 499/20000 nfe
[MainProcess/INFO] generation 10: 996/20000 nfe
[MainProcess/INFO] generation 15: 1496/20000 nfe
[MainProcess/INFO] generation 20: 1994/20000 nfe
[MainProcess/INFO] generation 25: 2492/20000 nfe
[MainProcess/INFO] generation 30: 2988/20000 nfe
[MainProcess/INFO] generation 35: 3486/20000 nfe
[MainProcess/INFO] generation 40: 3986/20000 nfe
[MainProcess/INFO] generation 45: 4486/20000 nfe
[MainProcess/INFO] generation 50: 4985/20000 nfe
[MainProcess/INFO] generation 55: 5483/20000 nfe
[MainProcess/INFO] generation 60: 5978/20000 nfe
[MainProcess/INFO] generation 65: 6478/20000 nfe
[MainProcess/INFO] generation 70: 6976/20000 nfe
[MainProcess/INFO] generation 75: 7472/20000 nfe
[MainProcess/INFO] generation 80: 7971/20000 nfe
[MainProcess/INFO] generation 85: 8470/20000 nfe
[MainProcess/INFO] generation 90: 8970/20000 nfe
[MainProcess/INFO] generation 95: 9468/20000

In [7]:
results, convergence = results_epsilon_0_1

In [8]:
# Save Results
save_results(results_epsilon_0_1, "../results/"+str(n_nfe)+"nfe_BaseCaseMORDM_Gelderland.tar.gz")

[MainProcess/INFO] results saved successfully to /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/20000nfe_BaseCaseMORDM_Gelderland.tar.gz


In [31]:
# Load Results
results_epsilon_0_1 = load_results("../results/"+str(n_nfe)+"nfe_BaseCaseMORDM_Gelderland.tar.gz")

[MainProcess/INFO] results loaded succesfully from /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/20000nfe_BaseCaseMORDM_Gelderland.tar.gz


### Check for Convergence
The optimization algorithm is now checked for convergence based on the Epsilon progress 

In [53]:
results, convergence = results_epsilon_0_1
convergence = pd.DataFrame(convergence)
fig = px.line(convergence, x = "nfe", y = "epsilon_progress", title="Epsilon Progress")
fig.show()

## Show tradeoffs in the resulting optimal policies
The algorithm found 973 solutions. 

In [55]:
fig = px.parallel_coordinates(results.iloc[:, 31::], labels= {"A1_2 Aggr Expected Annual Damage" : "A1 & 2 Damage", "A3 Expected Annual Damage" : "A3 Damage", "A1_2 Aggr Expected Number of Deaths" : "A1 & 2 Casualties", "A3 Aggr Expected Number of Deaths" : "A3 Casualties", "A1_5 Dike Investment Costs" : "Investment Costs", "Room for River Investment Costs" : "RfR Investment Costs"}, color=results.index, dimensions = ["A1_2 Aggr Expected Annual Damage", "A3 Expected Annual Damage", "A1_2 Aggr Expected Number of Deaths","A3 Aggr Expected Number of Deaths", "A1_5 Dike Investment Costs", "Room for River Investment Costs" ])
fig.update_layout(showlegend=False)
fig.show()

To show the tradeoffs more clearly, we aggregate the costs and the expected number of casualties, as well as the investment costs with the evacuation costs

In [12]:
results_agg = results.iloc[:, 31::]
results_agg["Expected Casualties"] = results_agg["A1_2 Aggr Expected Number of Deaths"] + results_agg["A3 Aggr Expected Number of Deaths"]
results_agg["Expected Damage"] = results_agg["A1_2 Aggr Expected Annual Damage"] + results_agg["A3 Expected Annual Damage"]
results_agg["Traditional Costs"] = results_agg["A1_5 Dike Investment Costs"] + results_agg["Evacuation Costs"]


In [114]:
fig = px.parallel_coordinates(results_agg, dimensions=["Expected Casualties","Expected Damage", "Traditional Costs", "Room for River Investment Costs"], color = results_agg.index)
fig.update_layout(showlegend=False)
fig.show()

## Re-Evaluate under deep uncertainty
For evaluating the promising policies under deep uncertainty, we set a hard limit on one expected casualties below 0.001 and expected damages below €100.000. 

In [14]:
results_subset = results[
    ((results["A1_2 Aggr Expected Number of Deaths"] + results["A3 Aggr Expected Number of Deaths"]) < 0.001) &
    ((results["A1_2 Aggr Expected Annual Damage"] + results["A3 Expected Annual Damage"]) < 100000) ]

We further use the remaining policies to test them under deep uncertainty by running the policies for 1000 scenarios each. 

In [15]:
policies = results_subset.iloc[:,0:31]
policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [16]:
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies_to_evaluate)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1000 scenarios * 18 policies * 1 model(s) = 18000 experiments
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 3600 cases completed
[MainProcess/INFO] 5400 cases completed
[MainProcess/INFO] 7200 cases completed
[MainProcess/INFO] 9000 cases completed
[MainProcess/INFO] 10800 cases completed
[MainProcess/INFO] 12600 cases completed
[MainProcess/INFO] 14400 cases completed
[MainProcess/INFO] 16200 cases completed
[MainProcess/INFO] 18000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [17]:
#Save results
save_results(results, "../results/DeepUncertainty8policies_"+str(n_scenarios)+"scenarios_Gelderland.tar.gz")

[MainProcess/INFO] results saved successfully to /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/DeepUncertainty8policies_1000scenarios_Gelderland.tar.gz


In [57]:
#Load results
results = load_results("../results/DeepUncertainty8policies_"+str(n_scenarios)+"scenarios_Gelderland.tar.gz")

[MainProcess/INFO] results loaded succesfully from /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/DeepUncertainty8policies_1000scenarios_Gelderland.tar.gz


### Signal-to-noise Ratio
To find the most robust policy, we rate the different policies by calculating the signal-to-noise ratio. 

In [18]:
def s_to_n(data, direction):
    mean = np.mean(data)
    std = np.std(data)
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [60]:
experiments, outcomes = results

overall_scores = {}
for policy in np.unique(experiments['policy']):
    scores = {}
    
    logical = experiments['policy']==policy
    
    for outcome in dike_model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
scores = pd.DataFrame.from_dict(overall_scores).T
scores = scores.reset_index()
scores

,index,A1_2 Aggr Expected Annual Damage,A3 Expected Annual Damage,A1_2 Aggr Expected Number of Deaths,A3 Aggr Expected Number of Deaths,A1_5 Dike Investment Costs,Room for River Investment Costs,Evacuation Costs
0,26,1.950930e+11,2.472914e+10,2.290124e-09,1.522982e-09,0.000000,0.0,1.269112e+05
1,120,5.736757e+13,1.247349e+11,3.369952e-07,7.914261e-09,0.000000,0.0,7.006888e+05
2,416,2.241515e+14,2.876282e+09,8.737306e-05,7.049526e-09,0.000000,0.0,0.000000e+00
3,417,1.849846e+15,2.731501e+10,1.886190e-05,1.785753e-09,3054.828216,0.0,2.600725e+05
4,419,5.573449e+13,5.152380e+09,1.998038e-07,2.559858e-10,0.000000,0.0,1.373088e+05
5,422,5.100384e+13,3.421490e+10,1.191449e-05,9.247041e-08,1013.169417,0.0,0.000000e+00
6,423,5.619190e+13,2.653684e+10,1.397581e-05,7.522977e-08,3806.437913,0.0,0.000000e+00
7,467,2.527578e+12,2.477728e+11,6.920938e-07,6.676501e-07,1100.948388,0.0,0.000000e+00
8,471,1.167043e+14,5.152380e+09,3.963251e-05,1.777679e-08,1730.115180,0.0,0.000000e+00
9,491,3.248518e+11,2.159940e+10,4.782207e-09,9.709301e-10,0.000000,0.0,8.427690e+04


In [62]:
fig = px.parallel_coordinates(scores, labels= {"A1_2 Aggr Expected Annual Damage" : "A1 & 2 Damage", "A3 Expected Annual Damage" : "A3 Damage", "A1_2 Aggr Expected Number of Deaths" : "A1 & 2 Casualties", "A3 Aggr Expected Number of Deaths" : "A3 Casualties", "A1_5 Dike Investment Costs" : "Investment Costs", "Room for River Investment Costs" : "RfR Investment Costs"}, color = scores.index, dimensions=["A1_2 Aggr Expected Annual Damage", "A3 Expected Annual Damage", "A1_2 Aggr Expected Number of Deaths","A3 Aggr Expected Number of Deaths", "A1_5 Dike Investment Costs", "Room for River Investment Costs" ])
fig.show()

### Maximum Regret
Another robustness criterion is the maximum regret measure. We again calculate this measure for every policy selected

In [63]:
def calculate_regret(data, best):
    return np.abs(best-data)


overall_regret = {}
max_regret = {}
for outcome in dike_model.outcomes:
    policy_column = experiments['policy']
    
    # create a DataFrame with all the relevent information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes[outcome.name], 
                         "policy":experiments['policy'],
                         "scenario":experiments['scenario']})
    
    data = data.pivot(index='scenario', columns='policy')
    
    data.columns = data.columns.get_level_values(1)
    
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()

In [106]:
max_regret = pd.DataFrame(max_regret)
max_regret = max_regret.reindex()

In [107]:

fig = px.imshow(max_regret/max_regret.max(), labels=dict(x="Outcomes", y = "Policies", color = "Maximum regret"))
fig.show()

In [109]:
fig = px.parallel_coordinates(max_regret, labels= {"A1_2 Aggr Expected Annual Damage" : "A1 & 2 Damage", "A3 Expected Annual Damage" : "A3 Damage", "A1_2 Aggr Expected Number of Deaths" : "A1 & 2 Casualties", "A3 Aggr Expected Number of Deaths" : "A3 Casualties", "A1_5 Dike Investment Costs" : "Investment Costs", "Room for River Investment Costs" : "RfR Investment Costs"}, color = max_regret.index, dimensions= ["A1_2 Aggr Expected Annual Damage", "A3 Expected Annual Damage", "A1_2 Aggr Expected Number of Deaths","A3 Aggr Expected Number of Deaths", "A1_5 Dike Investment Costs", "Room for River Investment Costs" ])
fig.show()

## Scenario Discovery
We perform Scenario Discovery using the PRIM-algorithm to discover such scenarios in which the robust policies fail. We focus on such scenarios where deaths and damages occur, and where the total costs are above 2.5 Billion €. 

In [153]:

outcomes["condition1"] = outcomes["A1_2 Aggr Expected Annual Damage"] + outcomes["A3 Expected Annual Damage"] + outcomes["A1_2 Aggr Expected Number of Deaths"] + outcomes["A3 Aggr Expected Number of Deaths"]

outcomes["condition2"] = outcomes["A1_5 Dike Investment Costs"] + outcomes["Evacuation Costs"] + outcomes["Room for River Investment Costs"]

condition1 = outcomes["condition1"] > 0 

condition2 = outcomes["condition2"] > 4e9

In [154]:
x = experiments.iloc[:,0:19]
y =  condition1 | condition2
prim_alg = prim.Prim(x,y, threshold=0.5)
box = prim_alg.find_box()

[MainProcess/INFO] 18000 points remaining, containing 8570 cases of interest
[MainProcess/INFO] mean: 0.5673076923076923, mass: 0.052, coverage: 0.06196032672112019, density: 0.5673076923076923 restricted_dimensions: 10


In [155]:
box.inspect_tradeoff()


alt.VConcatChart(...)

We select box 11, as it shows a good mixture between coverage and density. 

In [157]:
n_box = 11
box.inspect(n_box)


coverage    0.327888
density     0.494023
id                11
mass           0.316
mean        0.494023
res_dim            7
Name: 11, dtype: object

                      box 11              \
                         min         max   
A.1_pfail        8.68381e-05    0.899376   
A.2_pfail        0.000278511    0.936279   
A.5_pfail        0.000378219    0.942312   
discount rate 2   {2.5, 4.5}  {2.5, 4.5}   
A.4_pfail            0.09978    0.999303   
A.1_Bmax             42.5543     349.721   
A.3_Bmax              45.537     333.938   

                                                            
                                                 qp values  
A.1_pfail                       [-1.0, 0.2499681437999243]  
A.2_pfail                       [-1.0, 0.4284078674910726]  
A.5_pfail                       [-1.0, 0.4546806268703188]  
discount rate 2                [0.09850499422812108, -1.0]  
A.4_pfail                      [0.30090178171150644, -1.0]  
A.1_Bmax                 

In [159]:
box.select(n_box)
scens_in_box = experiments.iloc[box.yi]
outcomes_in_box = {k:v[box.yi] for k,v in outcomes.items()}
#Save results
save_results((scens_in_box, outcomes_in_box), '../results/mordm_'+str(n_box)+'_Gelderland.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/ricoherzog/OneDrive/Uni/2019-2020 WiSe Den Haag/Q4/EPA1361 Model Based Decision Making/05_Project/model-based-decision-making/results/mordm_11_Gelderland.tar.gz


# Multi-Scenario MORDM
Based on the results of scenario discovery, we now continue to select three of such scenarios based on the worst outcomes in terms of casualties, damages and total costs. We will take each of these scenarios as a reference scenario to run an optimization algorithm for the levers to choose. 